In [1]:
import re

In [2]:
import spacy

2024-05-08 14:57:32.490417: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import coreferee

In [4]:
file_path = 'journal_txt.txt'

def read_journal(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def parse_journal_entries(content):
    entries = []
    # Pattern to identify the start of a journal entry (author and date)
    entry_pattern = re.compile(r'by (Willa Silverman|Ben Berkman|Ethan Silverman), \w+ \d{1,2}, \d{4}')

    entries_start = [match.start() for match in re.finditer(entry_pattern, content)]

    for i in range(len(entries_start)):
        start = entries_start[i]
        end = entries_start[i + 1] if i + 1 < len(entries_start) else len(content)

        # Extract the whole entry content
        entry_content = content[start:end].strip()

        # Extracting and formatting the title, author, and date
        title_start = content.rfind('\n', 0, start)
        title_end = content.rfind('\n', 0, title_start)
        title = content[title_end:title_start].strip()

        author_date_line = content[start:content.find('\n', start)].strip()
        author, date = author_date_line.split(',', 1)
        author = author.replace('by ', '').strip()
        date = date.strip()

        # Removing the redundant author-date line from the entry content
        entry_text = entry_content.replace(author_date_line, "", 1).strip()

        # Splitting entry content and comments
        comments_start = re.search(r'\nComments\n', entry_text)
        if comments_start:
            comments = entry_text[comments_start.end():].strip()
            entry_text = entry_text[:comments_start.start()].strip()
        else:
            comments = ""

        entries.append({
            'title': title,
            'author': author,
            'date': date,
            'entry': entry_text,
            'comments': comments
        })

    return entries

In [5]:
journal_content = read_journal(file_path)
parsed_entries = parse_journal_entries(journal_content)

In [36]:
def remove_special_characters(input_string):
    # This pattern matches any character that is NOT a letter, a digit, or a space
    result = re.sub(r'[^a-zA-Z0-9 ]', ' ', input_string)
    return result

print(remove_special_characters(parsed_entries[19]['entry']))
text = remove_special_characters(parsed_entries[19]['entry'])

Back from the quick overnight trip to Baltimore  Had another CT scan of my abdomen  I am almost used to the yucky tasting  drink  ingested before the procedure  but still could not help spitting up afterwards the contrast dye injected into me  which filled my mouth with a metallic salty taste and my body with a warm and unpleasant rush   Then I met with Dr  Mehta  the radiation oncologist  who said that although the stomach abscess has not completely resolved  it has shrunk by about half and is producing less fluid than before   she felt that the better plan at this point would be to remove the stomach drain  keep me on two strong antibiotics and an anti fungal for 4 6 more weeks  and proceed to chemo  Of course  the risk still remains that the infection could rear its head during chemo  causing the chemo to  blow up   as Dr  Patel said   and in turn necessitating a temporary stoppage of chemo and or reinsertion of stomach drain  I will need to ask what the chances of this are  I certa

In [7]:
# Coreference resolution
coref_nlp = spacy.load('en_core_web_lg')
coref_nlp.add_pipe('coreferee')

In [ ]:
def coref_text(text):
    coref_doc = coref_nlp(text)
    resolved_text = ""

    for token in coref_doc:
        repres = coref_doc._.coref_chains.resolve(token)
        if repres:
            resolved_text += " " + " and ".join(
                [
                    t.text
                    if t.ent_type_ == ""
                    else [e.text for e in coref_doc.ents if t in e][0]
                    for t in repres
                ]
            )
        else:
            resolved_text += " " + token.text

    return resolved_text

ctext = coref_text(text)
print(ctext)

In [9]:
# Copied from https://github.com/Babelscape/rebel/blob/main/spacy_component.py
from spacy import Language, util
from spacy.tokens import Doc, Span
from transformers import pipeline
from typing import List
import re

def extract_triplets(text: str) -> List[str]:
    """
    parses the text to triplets
    1. Split the text into tokens
    2. If the token is <triplet>, <subj>, or <obj>, then set the current variable to the appropriate value
    3. If the token is not one of the above, then append it to the appropriate variable
    4. If the current variable is <subj>, then append the triplet to the list of triplets
    :param text: str - the text to be parsed
    :type text: str
    :return: A list of dictionaries.
    """

    triplets = []
    relation, subject, relation, object_ = "", "", "", ""
    text = text.strip()
    current = "x"

    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():

        if (token == "<triplet>"):

            current = "t"

            if (relation != ""):

                triplets.append(
                        {
                            "head": subject.strip(),
                            "type": relation.strip(),
                            "tail": object_.strip()
                            }
                        )
                relation = ""

            subject = ""

        elif (token == "<subj>"):

            current = "s"

            if (relation != ""):

                triplets.append(
                    {
                        "head": subject.strip(),
                        "type": relation.strip(),
                        "tail": object_.strip()
                        }
                    )

            object_ = ""

        elif (token == "<obj>"):

            current = "o"
            relation = ""

        else:

            if (current == "t"):

                subject += " " + token

            elif (current == "s"):

                object_ += " " + token

            elif (current == "o"):

                relation += " " + token

    if ((subject != "") and (relation != "") and (object_ != "")):

        triplets.append(
                {
                    "head": subject.strip(),
                    "type": relation.strip(),
                    "tail": object_.strip()
                    }
                )

    return triplets


@Language.factory(
        "rebel",
        requires = ["doc.sents"],
        assigns = ["doc._.rel"],
        default_config = {
            "model_name": "Babelscape/rebel-large",
            "device": 0,
            },
        )
class RebelComponent:

    def __init__(
            self,
            nlp, name,
            model_name: str,
            device: int,
        ):

        assert model_name is not None, ""

        self.triplet_extractor = pipeline(
                "text2text-generation",
                model = model_name,
                tokenizer = model_name,
                device = device
                )

        # Register custom extension on the Doc
        if (not Doc.has_extension("rel")):

            Doc.set_extension("rel", default = {})

    def _generate_triplets(self, sents: List[Span]) -> List[List[dict]]:
        """
        1. We pass the text of the sentence to the triplet extractor.
        2. The triplet extractor returns a list of dictionaries.
        3. We extract the token ids from the dictionaries.
        4. We decode the token ids into text.
        5. We extract the triplets from the text.
        6. We return the triplets.
        The triplet extractor is a model that takes a sentence as input and returns a list of dictionaries.
        Each dictionary contains the token ids of the extracted triplets.
        The token ids are the numbers that represent the words in the sentence.
        For example, the token id of the word "the" is 2.
        The token ids are decoded into text using the tokenizer.
        The tokenizer is a model that takes a list of token ids as input and returns a list of words.
        :param sents: List[Span]
        :type sents: List[Span]
        :return: A list of lists of dicts.
        """

        output_ids = self.triplet_extractor(
                [sent.text for sent in sents],
                return_tensors = True,
                return_text = False
                )  # [0]["generated_token_ids"]
        extracted_texts = self.triplet_extractor.tokenizer.batch_decode(
            [out["generated_token_ids"] for out in output_ids]
            )
        extracted_triplets = []

        for text in extracted_texts:

            extracted_triplets.extend(extract_triplets(text))

        return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        """
        The function takes a spacy Doc object and a list of triplets (dictionaries) as input.
        For each triplet, it finds the substring in the Doc object that matches the head and tail of the triplet.
        It then creates a spacy span object for each of the head and tail.
        Finally, it creates a dictionary of the relation type, head span and tail span and adds it to the Doc object
        :param doc: the spacy Doc object
        :type doc: Doc
        :param triplets: List[dict]
        :type triplets: List[dict]
        """

        text = doc.text.lower()

        for triplet in triplets:

            if (triplet["head"] == triplet["tail"]):

                continue

            head_match = re.search(
                r'\b' + re.escape(triplet["head"].lower()) + r'\b', text)
            if head_match:
                head_index = head_match.start()
            else:
                head_index = text.find(triplet["head"].lower())

            tail_match = re.search(
                r'\b' + re.escape(triplet["tail"].lower()) + r'\b', text)
            if tail_match:
                tail_index = tail_match.start()
            else:
                tail_index = text.find(triplet["tail"].lower())

            if ((head_index == -1) or (tail_index == -1)):

                continue

            head_span = doc.char_span(head_index, head_index + len(triplet["head"]), alignment_mode = "expand")
            tail_span = doc.char_span(tail_index, tail_index + len(triplet["tail"]), alignment_mode = "expand")

            try:

                offset = (head_span.start, tail_span.start)

            except (AttributeError):

                continue

            if (offset not in doc._.rel):

                doc._.rel[offset] = {
                        "relation": triplet["type"],
                        "head_span": head_span,
                        "tail_span": tail_span,
                        }

    def __call__(self, doc: Doc) -> Doc:
        """
        The function takes a doc object and returns a doc object
        :param doc: Doc
        :type doc: Doc
        :return: A Doc object with the sentence triplets added as annotations.
        """

        sentence_triplets = self._generate_triplets(doc.sents)
        self.set_annotations(doc, sentence_triplets)

        return doc

    def pipe(self, stream, batch_size = 128):
        """
        It takes a stream of documents, and for each document,
        it generates a list of sentence triplets,
        and then sets the annotations for each sentence in the document
        :param stream: a generator of Doc objects
        :param batch_size: The number of documents to process at a time, defaults to 128 (optional)
        """

        for docs in util.minibatch(stream, size=batch_size):

            sents = []

            for doc in docs:

                sents += doc.sents

            sentence_triplets = self._generate_triplets(sents)
            index = 0

            for doc in docs:

                n_sent = len(list(doc.sents))
                self.set_annotations(doc, sentence_triplets[index : index + n_sent])
                index += n_sent

                yield doc

In [10]:
nlp = spacy.load("en_core_web_lg")

nlp.add_pipe(
    "rebel",
    after="senter",
    config={
        "device": -1,  # Number of the GPU, -1 if want to use CPU
        "model_name": "Babelscape/rebel-large",
    },  # Model used, will default to 'Babelscape/rebel-large' if not given
)

/Users/benberkman/Documents/DS/journal-graph/graph/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

In [11]:
doc = nlp(ctext)
for value, rel_dict in doc._.rel.items():
    print(rel_dict)

{'relation': 'shares border with', 'head_span': New York, 'tail_span': Maryland}
{'relation': 'shares border with', 'head_span': Maryland, 'tail_span': New York}
{'relation': 'composer', 'head_span': La Valse, 'tail_span': Jacques Brel}
{'relation': 'followed by', 'head_span': And now we go around, 'tail_span': Again we go around}
{'relation': 'follows', 'head_span': Again we go around, 'tail_span': And now we go around}
{'relation': 'instance of', 'head_span': And now we spin around, 'tail_span': Carousel}
{'relation': 'performer', 'head_span': Seasons in the Sun, 'tail_span': Le Moribond}
{'relation': 'subclass of', 'head_span': P4tMl9, 'tail_span': watch}
{'relation': 'subclass of', 'head_span': fWi8, 'tail_span': watch}
{'relation': 'genre', 'head_span': Brel, 'tail_span': singing}
{'relation': 'different from', 'head_span': chemo regimen, 'tail_span': chemo}
{'relation': 'different from', 'head_span': chemo, 'tail_span': chemo regimen}
{'relation': 'field of work', 'head_span': MS

In [12]:
text

'1 377   1 901                     1 378   1 901 A brief update between a visit to family in New York  fun  if a bit arduous physically  but more on that in another posting   from which I returned yesterday  and a drive tomorrow to Maryland  for a telemedicine appointment with Dr  Beavis  There have been many  many twists and turns over the past few days as I try to share information with three oncologists  figure out the optimal order of meetings  and of course arrive at decisions regarding the best path forward  I m picturing myself making arrangements with Dr  Beavis  nurse while on the train home from NY yesterday  I try to speak as softly as possible  unlike the remarkable number of folks gabbing at full volume into their phones  often for hours  this actually led me to a semi altercation with a woman on the train  but I ll skip that story for now   and conversing with Dr  Cohen s nurse twice  once while a sitting in the big comfy chair in the infusion room  I got  stuck  in the c

In [10]:
import requests

def retrieve_wiki_id(item):
    try:
        url = "https://www.wikidata.org/w/api.php?action=wbsearchentities"
        params = f"&search={item}&language=en&format=json"
        data = requests.get(url + params).json()
        return {
            "id": data["search"][0]["url"],
            "description": data["search"][0]["display"]["description"]["value"],
        }
    except Exception as e:
        return None

In [11]:
entities = set()
for value, rel_dict in doc._.rel.items():
    entities.update([rel_dict["head_span"], rel_dict["tail_span"]])

for entity in entities:
    wiki_data = retrieve_wiki_id(entity)
    print(entity, wiki_data)

Apple I {'id': '//www.wikidata.org/wiki/Q18981', 'description': 'first computer model built by Apple'}
Apple Inc {'id': '//www.wikidata.org/wiki/Q312', 'description': 'American multinational technology company'}
Steve Wozniak {'id': '//www.wikidata.org/wiki/Q483382', 'description': 'American computer pioneer, inventor, computer engineer and programmer; co-founder of Apple Inc.'}
Semetric None
mark of the beast {'id': '//www.wikidata.org/wiki/Q6770514', 'description': 'album by Manilla Road'}
Steve Jobs {'id': '//www.wikidata.org/wiki/Q19837', 'description': 'American entrepreneur; co-founder of Apple Inc. (1955–2011)'}
Homebrew Computer Club {'id': '//www.wikidata.org/wiki/Q1195191', 'description': "computer hobbyist users' group in California"}
motherboard {'id': '//www.wikidata.org/wiki/Q4321', 'description': 'main printed circuit board (PCB) for a computing device'}
CPU {'id': '//www.wikidata.org/wiki/Q5300', 'description': 'electronic circuitry within a computer that carries out th